In [1]:
#using Pkg
#Pkg.add("GeneNetworkAPI")

In [2]:
using Helium;
using DataFrames;
using CSV;
using GeneNetworkAPI;

In [3]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/HSNIH-Palmer"

In [4]:
pheno_10441 = get_pheno("HSNIH-Palmer", "10441");

In [5]:
size(pheno_10441)

(4099, 4)

In [6]:
pheno_10441[1:6, :]

,data_id,sample_name,sample_name_2,value
,Int64,String,String,Float64
1,41013852,00071F4FAF,00071F4FAF,67.0
2,41013852,00071FB160,00071FB160,32.0
3,41013852,0007207A73,0007207A73,70.0
4,41013852,0007213055,0007213055,45.0
5,41013852,0007213615,0007213615,59.0
6,41013852,00072136BA,00072136BA,38.0


In [7]:
geno_filename = "HSNIH-Palmer.he";

In [8]:
geno_mat = readhe(geno_filename);

In [9]:
size(geno_mat)

(134918, 6147)

In [10]:
geno_supp = Helium.getsupp(geno_filename);
geno_supp[1:10, :]

10×5 Matrix{String}:
 "SNPID"  "rsid"          "position"  "alleleA"  "alleleB"
 "1"      "chr1:55365"    "55365"     "A"        "T"
 "1"      "chr1:666374"   "666374"    "T"        "C"
 "1"      "chr1:666382"   "666382"    "A"        "T"
 "1"      "chr1:666394"   "666394"    "G"        "A"
 "1"      "chr1:669529"   "669529"    "C"        "T"
 "1"      "chr1:669562"   "669562"    "T"        "A"
 "1"      "chr1:671466"   "671466"    "T"        "C"
 "1"      "chr1:759319"   "759319"    "T"        "C"
 "1"      "chr1:1134030"  "1134030"   "A"        "G"

In [11]:
geno_sampleID = Helium.getcolnames(geno_filename);

In [12]:
pheno_sampleID = pheno_10441.sample_name;

In [13]:
match_id = Array{Int, 1}(undef, 6147);

In [14]:
@time begin 
    for i in 1:6147
    
    indicator = 0;
    for j in 1:4099
        if geno_sampleID[i] == pheno_sampleID[j]
            indicator = indicator + 1;
        end
    end
    
    match_id[i] = indicator;
end
end

  2.142621 seconds (45.16 M allocations: 689.133 MiB, 14.34% gc time)


In [15]:
sum(match_id)

4094

In [16]:
typeof(match_id)

Vector{Int64} (alias for Array{Int64, 1})

In [17]:
samples_in_common = geno_sampleID[match_id .==1];

In [18]:
match_id_pheno = Array{Int, 1}(undef, 4099);

In [19]:
@time begin 
    for i in 1:length(match_id_pheno)
    
    indicator = 0;
    for j in 1:length(samples_in_common)
        if pheno_sampleID[i] == samples_in_common[j]
            indicator = indicator + 1;
        end
    end
    
    match_id_pheno[i] = indicator;
end
end

  2.091086 seconds (46.17 M allocations: 960.750 MiB)


In [20]:
sum(match_id_pheno)

4094

In [21]:
size(geno_mat)

(134918, 6147)

In [22]:
size(pheno_10441)

(4099, 4)

In [23]:
## subset geno:
geno_common = geno_mat[:, match_id .== 1];
## subset pheno:
pheno_common = pheno_10441[match_id_pheno .== 1, :];

In [24]:
size(pheno_common)

(4094, 4)

In [25]:
size(geno_common)

(134918, 4094)

Finally, we got a sample genofile with 4094 individuals, and 134918 genotype markers.

In [26]:
n = size(geno_common, 2)

4094

In [27]:
mean_freq = sum(geno_common; dims = 2)./(2*n);

In [28]:
less_common_markers = map(x -> (x > 0.95) || (x < 0.05), mean_freq)[:, 1];

In [29]:
typeof(less_common_markers)

Vector{Bool} (alias for Array{Bool, 1})

In [30]:
sum(less_common_markers)

16546

In [31]:
134918 - 16546

118372

In [32]:
geno_processed = permutedims(geno_common[less_common_markers .!= 1, :]);

In [33]:
pheno_processed = reshape(pheno_common[:, 4], :, 1);

In [34]:
size(geno_processed)

(4094, 118372)

In [35]:
size(pheno_processed)

(4094, 1)

## Run BulkLMM:

In [36]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using Plots
using Profile
using Distributed

In [37]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/HSNIH-Palmer"

In [38]:
include("../../src/scan.jl");
include("../../src/kinship.jl");
include("../../src/lmm.jl");
include("../../src/parallel_helpers.jl");
include("../../src/util.jl");
include("../../src/wls.jl");

In [39]:
@time kinship_BulkLMM = calcKinship(geno_processed);

LoadError: InterruptException:

In [43]:
size(kinship_BulkLMM)

LoadError: UndefVarError: kinship_BulkLMM not defined

In [44]:
pheno_common

,data_id,sample_name,sample_name_2,value
,Int64,String,String,Float64
1,41013852,00071F4FAF,00071F4FAF,67.0
2,41013852,00071FB160,00071FB160,32.0
3,41013852,0007207A73,0007207A73,70.0
4,41013852,0007213055,0007213055,45.0
5,41013852,0007213615,0007213615,59.0
6,41013852,00072136BA,00072136BA,38.0
7,41013852,0007213996,0007213996,36.0
8,41013852,0007213BC3,0007213BC3,57.0
9,41013852,00072140EF,00072140EF,47.0


In [42]:
@time scan(pheno_processed, geno_processed, kinship_BulkLMM; reml = false, method = "alt")

LoadError: UndefVarError: kinship_BulkLMM not defined

## Run Gemma:

In [36]:
gemma = "/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64";

In [37]:
run(`$gemma -h`)

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021

 type ./gemma -h [num] for detailed help
 options: 
  1: quick guide
  2: file I/O related
  3: SNP QC
  4: calculate relatedness matrix
  5: perform eigen decomposition
  6: perform variance component estimation
  7: fit a linear model
  8: fit a linear mixed model
  9: fit a multivariate linear mixed model
 10: fit a Bayesian sparse linear mixed model
 11: obtain predicted values
 12: calculate snp variance covariance
 13: note
 14: debug options

The GEMMA software is distributed under the GNU General Public v3
   -license    show license information
   see also http://www.xzlab.org/software.html, https://github.com/genetics-statistics


Process(`/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64 -h`, ProcessExited(0))